This file is a copied from a previous brackish water analysis file to update the baseline LCOW to the deep will injection scenarion in the Big Spring Case study

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import haversine as hs
import geopandas as gp
from shapely.geometry import Point, Polygon
import numpy as np

from pyomo.environ import Var, Expression, NonNegativeReals, Block, ConcreteModel, Constraint, Objective, Param, maximize, SolverFactory,RangeSet
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from pyomo.environ import Block, Expression, units as pyunits
import math
from watertap3.utils import watertap_setup, get_case_study, run_model 
from watertap3.utils import run_watertap3, run_model_no_print, run_and_return_model
from truck_pipe_cost_functions import truck_costing, pipe_costing

In [2]:
case_study = 'big_spring'
scenario = 'dwi_a'
desired_recovery = 1
ro_bounds = 'other' # or 'seawater'

m = watertap_setup(case_study=case_study, scenario=scenario)
m = get_case_study(m=m)

# run the baseline watertap3 version of this treatment train
m = run_watertap3(m, desired_recovery=desired_recovery, ro_bounds=ro_bounds)

m.fs.reverse_osmosis.membrane_area.unfix()
m.fs.reverse_osmosis.feed.pressure.unfix()

m = run_and_return_model(m, objective=True)

print(m.fs.costing.LCOW())


Case Study = BIG SPRING
Scenario = DWI A


=========================ADDING UNIT PROCESSES=========================
WELL FIELD
MICROFILTRATION
REVERSE OSMOSIS
UV AOP
TREATED STORAGE
MUNICIPAL DRINKING
DEEP WELL INJECTION


=========================START WT3 MODEL RUN==========================
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................

Case Study = BIG SPRING
Scenario = DWI A


=========================ADDING UNIT PROCESSES=========================
WELL FIELD
MICROFILTRATION
REVERSE OSMOSIS
UV AOP
TREATED STORAGE
MUNICIPAL DRINKING
DEEP WELL INJECTION

.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
...............................

In [5]:
m.fs.display()

Block fs

  Variables:
    None

  Objectives:
    objective_function : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 0.7585258937624717

  Constraints:
    None

  Blocks:
    Block fs.costing_param
    
      Variables:
        electricity_price : Electricity cost [$/kWh]
            Size=1, Index=None
            Key  : Lower : Value  : Upper : Fixed : Stale : Domain
            None :  None : 0.0545 :  None :  True :  True :  Reals
        maintenance_costs_percent_FCI : Maintenance/contingency cost as % FCI
            Size=1, Index=None
            Key  : Lower : Value : Upper : Fixed : Stale : Domain
            None :  None : 0.008 :  None :  True :  True :  Reals
        salaries_percent_FCI : Salaries cost as % FCI
            Size=1, Index=None
            Key  : Lower : Value  : Upper : Fixed : Stale : Domain
            None :  None : 0.0005 :  None :  True :  True :  Reals
        benefit_percent_of_salary : Benefits cost as % FCI
 

In [4]:
# Read in brackish database. It is filtered for the correct columns. 
df = pd.read_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/USGS_brackish_filtered.csv', index_col='unique_site_ID')

# convert USGS unit to our units: gpm to m3s, 
df.well_yield = df.well_yield * 6.30902e-5 #gpm to m3s
df.TDS_mgL = df.TDS_mgL / 1000 #mgL to kg/m3
df = df.rename(columns={'TDS_mgL': 'TDS_kgm3'})

path = 'C:/Users/mhardika/Documents/watertap3/WaterTAP3/watertap3/watertap3/data'

# add state look up for elec prices and add elec price column
state_abbv_df = pd.read_csv(path + '/state_abbv.csv', index_col='abbv')
elec_price_df = pd.read_csv(path + '/industrial_electricity_costs_2020.csv', index_col='location')
df['state'] = df.state_alpha.map(state_abbv_df.us_state)
df['elec_price'] = df.state.map(elec_price_df.cost)

# filter for well of interest in the US
df = df[df.well_yield >= 0.01]
df = df[df.TDS_kgm3 >= 0.5]
df = df[df.Latitude >= 22]
df = df[df.Longitude >= -150]
df = df[df.TDS_kgm3 <= 25]

# loop through brackish sites and calculate+save LCOW, capital, O&M, energy intensity, and recovery in a df.
capital = []
recovery = []
onm = []
lcow = []
pipe_cost = []
elec_int = []
tds_out = []
flow_in_list = []
tds_in_list = []
i = 0

for site in df.index:
    print('-----------------------------')
    print('SITE NAME:', site)
    i = i + 1
    print(i/len(df.index))
           
    m.fs.big_spring_feed.flow_vol_in.fix(df.loc[site].well_yield)
    m.fs.big_spring_feed.conc_mass_in[0, 'tds'].fix(df.loc[site].TDS_kgm3)
    m.fs.well_field.lift_height.fix(df.loc[site].well_depth_ft)
    
    m.fs.costing_param.electricity_price = df.loc[site].elec_price

    m = run_and_return_model(m, objective=True)

    temp = pipe_costing(m.fs.deep_well_injection.flow_vol_in[0].value*3600*24,10)*(1-m.fs.costing.system_recovery())
    
    flow_in_list.append(df.loc[site].well_yield)
    capital.append(m.fs.costing.capital_investment_total())
    recovery.append(m.fs.costing.system_recovery()*100)
    onm.append(m.fs.costing.operating_cost_annual())
    pipe_cost.append(temp)
    lcow.append(m.fs.costing.LCOW())
    elec_int.append(m.fs.costing.electricity_intensity())
    tds_out.append(m.fs.municipal_drinking.conc_mass_in[0, 'tds']())
    tds_in_list.append(df.loc[site].TDS_kgm3)
    
df['lcow'] = lcow
df['pipe_cost'] = pipe_cost
df['recovery'] = recovery
df['capital'] = capital
df['onm'] = onm
df['elec_int'] = elec_int
df['tds_out'] = tds_out

-----------------------------
SITE NAME: AZ_GEOTH-AZ0183
0.0001953506544246923
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0274
0.0003907013088493846
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0297
0.000586051963274077
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0327
0.0007814026176987692
.................................

Degrees of Freedom: 2

Initial solve attempt OPTIMAL

WaterTAP3 solution OPTIMAL

.................................
-----------------------------
SITE NAME: AZ_GEOTH-AZ0338
0.0009767532721234617
..

In [5]:
df.to_csv('/Users/mhardika/Documents/AMO/GeoToolAll_Methods/Water Source Data/Brackish/brackish_sites_with_metrics_baseline_dwi_updated_costs_transport_updated_basis.csv') 